# SpaceX Falcon 9 First Stage Landing Prediction

## EDA with SQL - Accessing DB with sqlmagic and analysing SpaceX Dataset
### Introduction

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore <b>if we can determine if the first stage will land, we can determine the cost of a launch.</b>

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


##### SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives flexibility of SQL.

### Connect to the database 

Let us first load the SQL extension and establish a connection with the database


In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

#connection object
con = sqlite3.connect("my_data1.db")

#cursor object
cur = con.cursor()

In [3]:
%sql sqlite:///my_data1.db

### Loading the Dataset

In [4]:
import pandas as pd
df = pd.read_csv(r"data_files/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Remove blank rows from table**


In [5]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [6]:
%sql select * from SPACEXTABLE LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Display the names of the unique launch sites  in the space mission


In [7]:
%sql select Distinct Launch_Site from SPACEXTABLE 

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


####  Display 5 records where launch sites begin with the string 'CCA' 


In [8]:
%sql select * from SPACEXTABLE where Launch_Site LIKE 'CCA%' LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Display the total payload mass carried by boosters launched by NASA (CRS)


In [9]:
%sql select sum(PAYLOAD_MASS__KG_) as TOTAL from SPACEXTABLE where Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


TOTAL
45596


### Display average payload mass carried by booster version F9 v1.1


In [10]:
%sql select avg(PAYLOAD_MASS__KG_) as Avg_of_payload_mass from SPACEXTABLE where Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


Avg_of_payload_mass
2928.4


### The first succesful ground landing date -

In [11]:
%sql select min(Date) as First_successful_landing_date from SPACEXTABLE where Landing_Outcome = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


First_successful_landing_date
2015-12-22


### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [12]:
%sql select Distinct Booster_Version from SPACEXTABLE \
where Landing_Outcome = 'Success (drone ship)' and (PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### List the total number of successful and failure mission outcomes


In [13]:
%sql select Mission_Outcome,count(*) as total_outcomes from SPACEXTABLE group by Mission_Outcome 

 * sqlite:///my_data1.db
Done.


Mission_Outcome,total_outcomes
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [14]:
%sql select count(Mission_Outcome) as total_success_outcomes from SPACEXTABLE where Mission_Outcome like 'Success%'

 * sqlite:///my_data1.db
Done.


total_success_outcomes
100


In [15]:
%sql select count(Mission_Outcome) as total_failure_outcomes from SPACEXTABLE where Mission_Outcome like 'Failure%'

 * sqlite:///my_data1.db
Done.


total_failure_outcomes
1


### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [16]:
%sql select Booster_Version,PAYLOAD_MASS__KG_ from SPACEXTABLE \
where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE) 

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


### List the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note:** SQLLite does not support monthnames. So we need to use `substr(Date, 6, 2)` as month to get the months and SUBSTR(Date, 1, 4)='2015' for year.

**SUBSTR(Date_string, starting position, for how many charaters)**


In [17]:
%sql select substr(Date,6,2) as Month, Landing_Outcome, Booster_Version, Launch_Site from SPACEXTABLE \
where Landing_Outcome = 'Failure (drone ship)' and SUBSTR(Date, 1, 4) = '2015'

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [18]:
%sql select Landing_Outcome, count(*) as Total_outcome,\
RANK() Over(order by count(*) DESC) as outcome_rank \
from SPACEXTABLE \
where Date >= '2010-06-04' AND Date <= '2017-03-20' \
Group by Landing_Outcome 


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Total_outcome,outcome_rank
No attempt,10,1
Success (ground pad),5,2
Success (drone ship),5,2
Failure (drone ship),5,2
Controlled (ocean),3,5
Uncontrolled (ocean),2,6
Precluded (drone ship),1,7
Failure (parachute),1,7
